In [13]:
import pandas as pd
train = pd.read_csv("train.tsv",sep="\t",index_col=0)
test = pd.read_csv("test.tsv",sep="\t",index_col=0)
sample_submit = pd.read_csv("sample_submit.csv",sep=",",index_col=0,header=None)

In [14]:
train.head()
#class = 0 左
#class = 1 平衡
#class = 2 右

,class,left_weight,left_distance,right_weight,right_distance
2,2,2,4,4,3
9,0,2,2,1,3
11,0,2,4,4,1
13,2,3,3,3,5
16,0,3,4,5,2


欠陥値やカテゴリ変数有無、外れ値がないか確認

In [15]:
train.describe()

,class,left_weight,left_distance,right_weight,right_distance
count,312.000000,312.000000,312.000000,312.000000,312.000000
mean,1.009615,3.025641,2.942308,2.987179,3.003205
std,0.957239,1.436541,1.444539,1.400446,1.444578
min,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.000000,2.000000,2.000000,2.000000
50%,1.000000,3.000000,3.000000,3.000000,3.000000
75%,2.000000,4.000000,4.000000,4.000000,4.000000
max,2.000000,5.000000,5.000000,5.000000,5.000000


目的変数に対しての各カラムの相関係数の確認

→特徴量が大きい列を学習に使う。その際、多重共線性にも注意


多重共線性とは、説明変数同士に相関があること。そのまま学習に使うと二重で学習されてしまう。

In [16]:
train.corrwith(train['class'])
#相関係数の解釈
#classを減らす要因としてleft_XXが関係している
#classを増やす要因としてright_XXが関係している
# 重さと距離については左右で係数が異なっているが現実世界では同じなはず


class             1.000000
left_weight      -0.374309
left_distance    -0.401885
right_weight      0.415045
right_distance    0.406904
dtype: float64

In [17]:
X = train.drop(['class'],axis=1)
y = train['class']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [19]:
import lightgbm as lgb
model = lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0)

In [20]:
eval_set = [(X_test, y_test)]
callbacks = []
callbacks.append(lgb.early_stopping(stopping_rounds=10))
callbacks.append(lgb.log_evaluation())
model.fit(X_train, y_train, eval_set=eval_set, callbacks=callbacks)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 24
[LightGBM] [Info] Number of data points in the train set

LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0)

In [21]:
from sklearn import metrics
y_pred = model.predict(test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.


In [22]:
sample_submit[1] = y_pred
sample_submit.to_csv('submit.tsv', header=None, sep='\t')

In [25]:
y_pred

array([0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0,
       2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0,
       2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2,
       2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0,
       0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2,
       0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2,
       0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2,
       0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2,
       0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2,
       2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0,
       0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2,

In [24]:
sample_submit

,1
0,
0,0
1,2
3,2
4,2
5,2
...,...
619,0
620,2
621,2
